# An Introduction to LROSE

<img align="right" width="200" height="200" src="../images/colette_rose.jpg">
<img align="right" width="200" height="200" src="../images/LROSE_logo_small.png">
    
The Lidar Radar Open Software Environment (LROSE) is an National Science Foundation (NSF) supported project to develop common software for the Lidar, Radar, and Profiler community based on collaborative, open source development.  The core package is being jointly developed by Colorado State University (CSU) and the Earth Observing Laboratory at the NSF National Center for Atmospheric Research (NSF NCAR/EOL). The current LROSE release is called Colette (a versatile climbing rose). 

More information on LROSE can be found on the [LROSE wiki](http://wiki.lrose.net/index.php/Main_Page).

(add LROSE forum, contact info, link to listserv)


## Notebook Summary

This notebook will cover the following:

1. LROSE Overview
2. Basic LROSE application command line usage
3. Example LROSE application parameter files
4. Basic data inspection
5. Converting a file to CfRadial
6. Simple gridding
7. Basic plotting

## LROSE Overview

LROSE encompasses six key toolsets that define a core lidar/radar workflow: Convert, Display, QC, Grid, Echo, and Winds. Colette focuses on high-quality, well-tested, well-maintained and well-documented key applications as ‘building blocks’, allowing users to assemble trusted, reproducible workflows to accomplish more complex scientific tasks.

### Installation

LROSE is available for download through [GitHub](https://github.com/NCAR/lrose-core/). Installation is supported for Linux (source, packages) and Mac OS (source, Homebrew). Conda-forge development is underway.

### gateway advertisement

link to tutorials, lrose-hub, etc

### bugs slash questions

github issues, email mike.... (not ideal), LROSE forum

### Initialize python

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import datetime
import fsspec
# import glob
# import pytz
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.ticker as plticker
# from matplotlib.lines import Line2D
# import cartopy.crs as ccrs
# import cartopy.io.shapereader as shpreader
# import cartopy.geodesic as cgds
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
# from cartopy import feature as cfeature
# import shapely
# import netCDF4 as nc
# import pyart

# Set directory variable to call LROSE
os.environ['LROSE_DIR'] = '/usr/local/bin'
# os.environ['LROSE_DIR'] = '/usr/local/lrose/bin'
#base_dir = os.environ['BASE_DIR']

# Set the URL and path to access the data on the cloud
URL = "https://js2.jetstream-cloud.org:8001/"
path = f"pythia/radar/erad2024"

fs = fsspec.filesystem("s3", anon=True, client_kwargs=dict(endpoint_url=URL))

fs.glob(f"{path}/20240522_MeteoSwiss_ARPA_Lombardia/Data/Cband/*")

files = fs.glob(f"{path}/20240522_MeteoSwiss_ARPA_Lombardia/Data/Cband/*.nc") #### FIX PATH
local_files = [
    fsspec.open_local(
        f"simplecache::{URL}{i}", s3={"anon": True}, filecache={"cache_storage": "."}
    )
    for i in files
]


In [ ]:
# LROSE prefers files to have .nc extension, so rename one file for this notebook
os.rename(local_files[0],local_files[0]+'.nc')

# set an environment variable so command line and python play nicely
os.environ['file_name'] = local_files[0]+'.nc'


In [ ]:
!pwd

## Notes on LROSE Parameter files

All LROSE applications have a detailed parameter file, which is read in at startup. The parameters allow the user to control the processing in the LROSE apps. To generate a default parameter file, you use the -print_params option for the app.

For example, for RadxConvert you would use:

```
  RadxConvert -print_params > RadxConvert.nexrad
```

and then edit RadxConvert.nexrad appropriately.

At runtime you would use:

```
  RadxConvert -params RadxConvert.nexrad ... etc ...
```

## View the RadxConvert parameter file

Note that we can use environment variables in the parameter files.

Environment variables are inserted using the format:

```
  $(env_var_name)
```

For example:

```
  input_dir = "$(NEXRAD_DATA_DIR)/raw/$(RADAR_NAME)";
```


In [ ]:
# View the param file
!cat $BASE_DIR/params/erad/RadxConvert.nexrad

In [ ]:
# command line options
!${LROSE_DIR}/RadxPrint -h

In [ ]:
# view the default parameter file
!${LROSE_DIR}/RadxPrint -print_params

In [ ]:
# save the default parameters to a file
!${LROSE_DIR}/RadxPrint -print_params > ./RadxPrint_params

# if you have an existing parameter file and want to update it (e.g., a new LROSE version is released),
# you could run a command similar to the following, just update the parameter file names
# !${LROSE_DIR}/RadxPrint -f ./path/to/RadxPrint_params -print_params > ./RadxPrint_params_new

## Inspect the data

In [ ]:
!${LROSE_DIR}/RadxPrint -f $file_name

In [ ]:
!${LROSE_DIR}/RadxPrint -summary -f $file_name | tail -200

## Format Conversion

In [ ]:
# Convert file (already cfradial, but you get the idea)
# this will create a new subdirectory called convert along with a subdirectory YYYYMMDD that contains the file
!${LROSE_DIR}/RadxConvert -f $file_name -outdir ./convert


## Simple Gridding

In [ ]:
!${LROSE_DIR}/Radx2Grid -f ./convert/*/*.nc -outdir ./gridded

## Simple Display

### Py-Art

In [ ]:
# Read CfRadial file into radar object
filePath = os.path.join('./', "convert/20240522/*.nc")
radar = pyart.io.read_cfradial(filePath)
radar.info('compact')

In [ ]:
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(1, (10, 10))

# DBZ

axDbz = fig.add_subplot(221)
display.plot_ppi('reflectivity', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
display.plot_range_rings([50, 100, 150, 200])
display.plot_cross_hair(200.)

# velocity 

axKdp = fig.add_subplot(222)
display.plot_ppi('velocity', 0, vmin=-15, vmax=15,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="velocity (m/s)")
display.plot_range_rings([50, 100, 150, 200])
display.plot_cross_hair(200.)

# spectrum width

axHybrid = fig.add_subplot(223)
display.plot_ppi('spectrum_width', 0,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="sw (m/s)")
display.plot_range_rings([50, 100, 150, 200])
display.plot_cross_hair(200.)

# NCAR PID (computed)

axPID = fig.add_subplot(224)
display.plot_ppi('differential_reflectivity', 0,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR (dB)")
display.plot_range_rings([50, 100, 150, 200])
display.plot_cross_hair(200.)

fig.tight_layout()
plt.show()

### Gridded Data

In [ ]:
# Read in example radar mosaic for a single time

filePathGridded = os.path.join('./', 'gridded/20240522/*.nc')
dsGridded = nc.Dataset(filePathGridded)
print("Radar mosaic file path: ", filePathGridded)
print("Radar mosaic data set: ", dsGridded)

# Compute time

uTimeSecs = dsGridded['start_time'][0]
startTime = datetime.datetime.fromtimestamp(int(uTimeSecs))
startTimeStr = startTime.strftime('%Y/%m/%d-%H:%M:%S UTC')
print("Start time: ", startTimeStr)

In [ ]:
# Create map for plotting lat/lon grids
def new_map(fig):
    
    ## Create projection centered on data
    proj = ccrs.PlateCarree()

    ## New axes with the specified projection:
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    
    ## Set extent the same as radar mosaic
    ax.set_extent([minLonMosaic, maxLonMosaic, minLatMosaic, maxLatMosaic])

    ## Add grid lines & labels:
    gl = ax.gridlines( crs=ccrs.PlateCarree()
                     , draw_labels=True
                     , linewidth=1
                     , color='lightgray'
                     , alpha=0.5, linestyle='--'
                     ) 
    gl.top_labels = False
    gl.left_labels = True
    gl.right_labels = False
    gl.xlines = True
    gl.ylines = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 8, 'weight': 'bold'}
    gl.ylabel_style = {'size': 8, 'weight': 'bold'}
    
    return ax

In [ ]:
# Plot column-max reflectivity
figDbzComp = plt.figure(figsize=(8, 8), dpi=150)
axDbzComp = new_map(figDbzComp)
plt.imshow(dbzPlaneMax,
            cmap='pyart_Carbone42',
            interpolation = 'bilinear',
            origin = 'lower',
            extent = (minLonMosaic, maxLonMosaic, minLatMosaic, maxLatMosaic))
axDbzComp.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
axDbzComp.add_feature(cfeature.STATES, linewidth=0.3, edgecolor='brown')
#axDbzComp.coastlines('10m', 'darkgray', linewidth=1, zorder=0)
plt.colorbar(label="DBZ", orientation="vertical", shrink=0.5)
plt.title("Radar DBZ: " + startTimeStr)

### Hawkeye

add in pre-canned or command line image

# Afternoon preview

Want to learn how to run multi-Doppler analysis? Join us in the afternoon!

### Additional resources

More tutorials can be found on our Gateway GitHub repo: 
https://github.com/nsf-lrose/lrose-hub

The LROSE wiki con 
forum